In [29]:
import torch
from transformers import AutoModel, AutoTokenizer
from vncorenlp import VnCoreNLP

In [33]:
import pandas as pd
import sys
import numpy as np

import operator
from models import Input, AspectOutput
from tqdm import tqdm
import re
from gensim.models import KeyedVectors
import json


def load_aspect_data_mebe(path):
    """

    :param path:
    :return:
    :rtype: (list of models.Input, list of models.AspectOutput)
    """
    inputs = []
    outputs = []
    df = pd.read_csv(path, encoding='utf-8')
    for _, r in df.iterrows():
        t = str(r['text']).strip()
        inputs.append(t)
        labels = list(range(10))
        scores = [0 if r['aspect{}'.format(i)] == 0 else 1 for i in range(1, 8)]
        outputs.append(scores)
    outputs = np.array(outputs)
    return inputs, outputs
inputs_shopee, outputs_shopee = load_aspect_data_mebe('data/raw_data/mebe_shopee.csv')
inputs_tiki, outputs_tiki = load_aspect_data_mebe('data/raw_data/mebe_tiki.csv')
inputs = inputs_shopee+inputs_tiki


In [34]:
len(inputs)

6004

In [35]:
import sys
import pickle
import tensorflow as tf
import numpy as np

In [36]:

# Call for PhoBERT pretrained model from huggingface-transformers
phoBert = AutoModel.from_pretrained('vinai/phobert-base', output_hidden_states=True)
tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base', use_fast=False)

# Load and use RDRSegmenter from VnCoreNLP as recommended by PhoBERT authors
# rdrsegmenter = VnCoreNLP(r"C:\Users\acer\Documents\hoc tap\KT LAB\scikit-learn\VnCoreNLP-master\VnCoreNLP-1.1.1.jar",
#                          annotators='wseg', max_heap_size='-Xmx500m')

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [37]:
inputs[0]

'mua đóng_gói cẩn_thận hình_ảnh t / c kiếm xu'

In [26]:
# x = rdrsegmenter.tokenize(inputs[0].text)

In [28]:
a = []
for x in x:
        a.append(' '.join(x))
a

['mua đóng_gói cẩn_thận hình_ảnh t / c kiếm xu']

In [5]:
# segmented_texts = []  # Texts which are segmented
# # for inp in inputs[:10]:
# for inp in inputs:
#     _inp = rdrsegmenter.tokenize(inp.text)
#     for sen in _inp:
#         segmented_texts.append(' '.join(sen))

In [39]:
segmented_texts = inputs

In [40]:
encoded_texts = []  # Texts are converted to indices vectors with 2 added token indices for <s>, </s>
for st in segmented_texts:
    _st = tokenizer.encode(st)
    encoded_texts.append(_st)

In [41]:
len(encoded_texts)

6004

In [42]:
maxlen = 0
for i in encoded_texts:
    if len(i) > maxlen: maxlen = len(i)
maxlen


201

In [43]:
encoded_texts[0]

[0, 188, 8179, 3289, 425, 1204, 95, 1894, 1095, 7044, 2]

In [44]:
masked_pos = []
for mp in encoded_texts:
    m = [int(token_id > 0) for token_id in mp]
    masked_pos.append(m)

In [45]:
tensors, masks = [], [] # Convert list of indices to torch tensor
for i in range(len(masked_pos)):
    tensors.append(torch.tensor([encoded_texts[i]]))
    masks.append(torch.tensor([masked_pos[i]]))

In [46]:
lhs = []  # There are 13 tensors of 13 attention layers from PhoBERT <=> 1 word has 13 (768,)-tensor
for i in range(len(tensors)):
    with torch.no_grad():
        f = phoBert(tensors[i], masks[i])
        hs = f[2]  # Len: 13 as 13 output tensors from 13 attention layers
        _hs = np.squeeze(np.array([x.detach().numpy() for x in hs]), axis=1)    # Reduce the dimension
        lhs.append(_hs)

In [47]:
# lhs[50].shape

In [48]:
reshaped_lhs = []   # Shape: num_words * 13 * 768
for rlhs in lhs:
    _rlhs = []
    for i in range(rlhs.shape[1]):
        a = np.array([x[i] for x in rlhs])
        _rlhs.append(a)
    reshaped_lhs.append(_rlhs)

In [40]:
# len(reshaped_lhs[0][0])

13

In [49]:
texts_token_emb = []
for tte in reshaped_lhs:
    _tte = []
    for i in tte:
        emb = tf.reduce_sum(i[-4:], axis=0)
        _tte.append(emb)
    texts_token_emb.append(np.array(_tte[1:-1]))
    

In [50]:
np.save('phobert_embed', texts_token_emb)

In [16]:
phobert_embed = np.load('phobert_embed.npy', allow_pickle=True)

In [17]:
phobert_embed[40].shape

(9, 768)

In [55]:
texts_token_emb[0]

array([[-3.0757074 ,  2.2905777 , -0.9053175 , ..., -0.04593234,
        -2.077567  ,  1.2567792 ],
       [-1.5911925 ,  2.013134  , -0.57891965, ...,  0.49071312,
        -0.953002  , -0.48284584],
       [-0.0750351 ,  2.0042949 ,  0.21631894, ..., -0.25082076,
        -0.511614  ,  0.7755858 ],
       ...,
       [-0.7758073 ,  0.42336088, -0.21910533, ...,  0.24243325,
        -0.41877595,  1.0976824 ],
       [-0.36669153,  0.6132056 , -0.02825534, ...,  0.16419297,
        -1.3461001 ,  0.4586584 ],
       [-1.7559828 ,  1.8148768 ,  0.595081  , ..., -0.89722955,
        -0.6953317 ,  0.30204165]], dtype=float32)

In [43]:
s=[1,2,3,4]
s[1:-1]

[2, 3]

In [8]:
maxlen=0
for i in phobert_embed:
    if len(i) > maxlen: maxlen = len(i)
print(maxlen)

250


In [7]:
len(phobert_embed[0])

15